# Capstone Project: Quantum Wordle

In [134]:
# Every answer and guess has to contain this many letters/characters
WORD_LENGTH = 5

# Colour Feedback chars: Indicate correctness of corresponding letter in guess word
RIGHT_LETTER_RIGHT_SPOT_COLOUR = '🟩'
RIGHT_LETTER_WRONG_SPOT_COLOUR = '🟨'
WRONG_LETTER_COLOUR = '🟥'


def check_guess_correctness(guess_str, answer_str, word_length=WORD_LENGTH):
    """ Compares the guess with the answer and returns colour feedback indicating how close the guess was.

    Input:
        - guess_str: Guess word input by the user
        - answer_str: Answer word
        Note: 
            - Assumes both input strings are the same 'case', so they can be compared
            - Assumes both input strings consist of word_length characters

    Output:
        Returns a string consisting of word_length coloured boxes, where each box indicates the correctness of the corresponding letter of the guess word
    """

    # The guess is the same as the answer -- i.e. all the letters of the guess word are both the right letter and in the right position
    if guess_str == answer_str:
        # Assuming word_length = 5 and RIGHT_LETTER_RIGHT_SPOT_COLOUR = '🟩':
        # '🟩🟩🟩🟩🟩'
        colour_feedback_str = word_length * RIGHT_LETTER_RIGHT_SPOT_COLOUR
        return colour_feedback_str
    
    else:
        # Convert both guess and answer from string to list
        guess_char_list = list(guess_str)
        answer_char_list = list(answer_str)

        # This is technically a range object, not a list but, if it were converted to a list, if would look like this (assuming word_length = 5):
        # [0, 1, 2, 3, 4]
        word_index_list = range(word_length)

        # Each element of this list will be a coloured box character, where each box indicates the correctness of the corresponding letter of the guess word
        # Assuming word_length = 5:
        # [None, None, None, None, None]
        colour_feedback_list = [None for i in word_index_list]

        # NOTE: If there are repeated letters in either guess or answer, looking for a match (for each letter of guess) by iterating over the letters of answer from left to right is NOT guaranteed to work!
        # Eg. Let guess be 'CABAL' and 
        #        answer be 'ABBEY':
        #     In this case, if we do left to right evaluation of answer for each letter of guess, we will compare the 'B' in 'CABAL' to the 1st 'B' in 'ABBEY' and say that it is the right letter but the wrong position and then stop iterating over answer since we already found a match for that guess letter.
        #     However, the correct answer is to say that, since the 'B' in 'CABAL' matches the 2nd 'B' in 'ABBEY', it is actually both the right letter and the right position!

        # Check for right letter and right position matches by comparing each guess letter to the answer letter in the same position only
        # Note:
        #   - We are iterating over the guess and answer simultaneously, one time -- i.e. we are comparing 1st letter of guess to 1st letter of answer, 2nd letter of guess to 2nd letter of answer, etc.
        #   - We check this condition first, since this dominates in case of repeated guess letters (see example above)
        #   - Although not strictly necessary (since we're not changing the length of the answer_char_list, just its current element), to be safe, we iterate over a COPY of answer_char_list rather than the original answer_char_list itself (which is what we're changing during the iteration)
        for index, guess_char, answer_char in zip(word_index_list, guess_char_list, answer_char_list[:]):
            # Since we are only comparing guess and answer letters in the same position, if they match, then we already know that it's both the right letter and the right position
            if guess_char == answer_char:
                colour_feedback_list[index] = RIGHT_LETTER_RIGHT_SPOT_COLOUR
                # Now that a char in answer has been matched by a char in guess, remove that char from answer so that other chars in guess don't accidentally match it
                # Specifically, to avoid changing the length of answer_char_list, replace current char with None since that's guaranteed not to match any letter)
                answer_char_list[index] = None

        # At this point, we've identified all the letters of the guess that are both the right letter and in the right position.
        # The remaining letters are either the right letter but in the wrong position, or just the wrong letter.
        # At this point, for each guess letter, iterating over the answer letters from left to right WILL give us the right answer
        # Eg. Let guess be 'EERIE' and
        #        answer be 'TENET'
        #     In this case, assuming we previously found that the 2nd 'E' in 'EERIE' was both the right letter and in the right position, now we would expect to mark the 1st 'E' as being the right letter but in the wrong position (compared to the 2nd 'E' in 'TENET'), and the 3rd 'E' as being the wrong letter (no corresponding letter in 'TENET') -- i.e. after the 2nd 'E' in the guess is perfectly matched above, the LEFTMOST remaining 'E' in the guess is compared with the corresponding letter in the answer before the more rightward remaining 'E' in the guess.

        # Check for right letter but wrong position matches by comparing each guess letter to EVERY answer letter
        # For efficiency, only check the guess letters whose correctness is still unknown
        for guess_index, guess_char in enumerate(guess_char_list):
            # Correctness of this guess letter is still unknown
            if colour_feedback_list[guess_index] is None:
                guess_char_in_answer = False
                # Technically, we only need to compare each guess letter to all the answer letters that are NOT in the same position (since we already compared against the answer letter in the same position above), but it's simpler to just compare against every answer letter
                # Can safely compare guess letter against every answer letter without worrying about matching a previously matched answer letter, since the answer letters that were previously matched above have been replaced with None, which is guaranteed not to match any letter in guess
                for answer_index, answer_char in enumerate(answer_char_list[:]):
                    # Guess letter is the right letter but we already know it's in the wrong position (since otherwise we would have matched it above)
                    if guess_char == answer_char:
                        guess_char_in_answer = True
                        colour_feedback_list[guess_index] = RIGHT_LETTER_WRONG_SPOT_COLOUR
                        # Again, replace the matched answer letter with None so it doesn't accidentally get matched again later on by another (duplicate) guess letter
                        answer_char_list[answer_index] = None
                        # Now that we've found a match for the current guess letter, stop iterating through answer so that we don't wrongly match another (duplicate) answer letter and replace that with None as well!
                        # This ensures that one guess letter does not accidentally "use up" (match with) multiple duplicate answer letters
                        break
                # At this point, have checked guess_char against every letter in answer -- if we still haven't found a match, then guess_char is simply the wrong letter
                if not guess_char_in_answer:
                    colour_feedback_list[guess_index] = WRONG_LETTER_COLOUR

        # Now that we've assembled the colour feedback list for all the letters in the guess word, convert it from list to string and return it
        colour_feedback_str = ''.join(colour_feedback_list)
        return colour_feedback_str


def test_check_guess_correctness():
    """Used to quickly test check_guess_correctness()"""

    # Each tuple contains the input guess word, the answer word it will be compared against and the expected output colour feedback indicating how close/correct the guess was
    test_value_tuples = \
        [
            # Guess and answer are identical
            (
                'TWINS', 
                'TWINS', 
                f'{RIGHT_LETTER_RIGHT_SPOT_COLOUR}{RIGHT_LETTER_RIGHT_SPOT_COLOUR}{RIGHT_LETTER_RIGHT_SPOT_COLOUR}{RIGHT_LETTER_RIGHT_SPOT_COLOUR}{RIGHT_LETTER_RIGHT_SPOT_COLOUR}'
            ),

            # Guess and answer have no letters in common
            (
                'FRAUD', 
                'TWINS', 
                f'{WRONG_LETTER_COLOUR}{WRONG_LETTER_COLOUR}{WRONG_LETTER_COLOUR}{WRONG_LETTER_COLOUR}{WRONG_LETTER_COLOUR}'
            ),

            # Neither guess nor answer has repeated letters: Output contains all 3 match possibilities
            (
                'SWORE', 
                'WEARY', 
                f'{WRONG_LETTER_COLOUR}{RIGHT_LETTER_WRONG_SPOT_COLOUR}{WRONG_LETTER_COLOUR}{RIGHT_LETTER_RIGHT_SPOT_COLOUR}{RIGHT_LETTER_WRONG_SPOT_COLOUR}'
            ),


            # Guess has repeated letters, but answer doesn't: 1st repeated letter of guess matches and is in same position as answer letter
            (
                'WEEPY', 
                'WEARY', 
                f'{RIGHT_LETTER_RIGHT_SPOT_COLOUR}{RIGHT_LETTER_RIGHT_SPOT_COLOUR}{WRONG_LETTER_COLOUR}{WRONG_LETTER_COLOUR}{RIGHT_LETTER_RIGHT_SPOT_COLOUR}'
            ),
            # Guess has repeated letters, but answer doesn't: 2nd repeated letter of guess matches and is in same position as answer letter
            (
                'EERIE', 
                'WEARY', 
                f'{WRONG_LETTER_COLOUR}{RIGHT_LETTER_RIGHT_SPOT_COLOUR}{RIGHT_LETTER_WRONG_SPOT_COLOUR}{WRONG_LETTER_COLOUR}{WRONG_LETTER_COLOUR}'
            ),


            # Both guess and answer have same repeated letters: 2nd repeated letter of guess matches and is in same position as 2nd of answer
            (
                'LEVER', 
                'EATEN', 
                f'{WRONG_LETTER_COLOUR}{RIGHT_LETTER_WRONG_SPOT_COLOUR}{WRONG_LETTER_COLOUR}{RIGHT_LETTER_RIGHT_SPOT_COLOUR}{WRONG_LETTER_COLOUR}'),
            # Both guess and answer have same repeated letters: 1st repeated letter of guess matches and is in same position as 2nd of answer
            (
                'KEBAB', 
                'ABBEY', 
                f'{WRONG_LETTER_COLOUR}{RIGHT_LETTER_WRONG_SPOT_COLOUR}{RIGHT_LETTER_RIGHT_SPOT_COLOUR}{RIGHT_LETTER_WRONG_SPOT_COLOUR}{RIGHT_LETTER_WRONG_SPOT_COLOUR}'
            ),
            # Both guess and answer have same repeated letters: Although they match, neither repeated letter of guess is in same position as corresponding repeated letters of answer
            (
                'PAPAL', 
                'ALARM', 
                f'{WRONG_LETTER_COLOUR}{RIGHT_LETTER_WRONG_SPOT_COLOUR}{WRONG_LETTER_COLOUR}{RIGHT_LETTER_WRONG_SPOT_COLOUR}{RIGHT_LETTER_WRONG_SPOT_COLOUR}'
            ),


            # Both guess and answer have same repeated letters, but guess has more than 2: The 3 repeated letters of guess span all three match possibilities
            (
                'EERIE', 
                'TENET', 
                f'{RIGHT_LETTER_WRONG_SPOT_COLOUR}{RIGHT_LETTER_RIGHT_SPOT_COLOUR}{WRONG_LETTER_COLOUR}{WRONG_LETTER_COLOUR}{WRONG_LETTER_COLOUR}'
            ),
            

            # Guess doesn't have repeated letters, but answer does: Guess letter matches and is in same position as 1st repeated letter of answer
            (
                'WEARY', 
                'WEEPY', 
                f'{RIGHT_LETTER_RIGHT_SPOT_COLOUR}{RIGHT_LETTER_RIGHT_SPOT_COLOUR}{WRONG_LETTER_COLOUR}{WRONG_LETTER_COLOUR}{RIGHT_LETTER_RIGHT_SPOT_COLOUR}'
            ),
            # Guess doesn't have repeated letters, but answer does: Guess letter matches and is in same position as 2nd repeated letter of answer
            (
                'WEARY', 
                'EERIE', 
                f'{WRONG_LETTER_COLOUR}{RIGHT_LETTER_RIGHT_SPOT_COLOUR}{WRONG_LETTER_COLOUR}{RIGHT_LETTER_WRONG_SPOT_COLOUR}{WRONG_LETTER_COLOUR}'
            )
        ]

    for guess_str, answer_str, expected_colour_feedback_str in test_value_tuples:
        actual_colour_feedback_str = check_guess_correctness(guess_str, answer_str)
        if actual_colour_feedback_str == expected_colour_feedback_str:
            print('Pass')
        else:
            print('Fail!')
            print('\tGuess:\t\t{}'.format('\t'.join(guess_str)))
            print('\tAnswer:\t\t{}'.format('\t'.join(answer_str)))
            print('\tExpected:\t{}'.format('\t'.join(expected_colour_feedback_str)))
            print('\tActual:\t\t{}'.format('\t'.join(actual_colour_feedback_str)))

# # Uncomment to run test suite
# test_check_guess_correctness()

Pass
Pass
Pass
Pass
Pass
Pass
Pass
Pass
Pass
Pass
Pass
